# Create Trip Transaction  Fact

In [0]:
from delta.tables import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F

In [0]:
# Azure Blob Storage details
storage_account_name = "<storage-account-name>"
storage_account_key = "<storage-account-access-key>"
container_name = "deltalake"
file_path1 = "bronze/Trip_Transactions"
file_path2 = "bronze/Rewards_Points"

In [0]:
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

In [0]:
df1=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path1}")

In [0]:
df1.show()

+-------+--------------------+--------------------+---------+-----------+------------------------+-----------+---------------------+--------------+-----------------------------+----------------+--------------------------+-------------------+--------------+----------+-----------+---------------------+--------------------+--------------------+-----------+-------------+
|trip_id|trip_start_timestamp|  trip_end_timestamp|driver_id|driver_name|source_location_address1|source_city|source_province_state|source_country|destination_location_address1|destination_city|destination_province_state|destination_country|total_distance|total_fare|trip_status|delay_start_time_mins|      payment_method|      payment_Status|customer_id|customer_name|
+-------+--------------------+--------------------+---------+-----------+------------------------+-----------+---------------------+--------------+-----------------------------+----------------+--------------------------+-------------------+--------------+----

In [0]:
df2=df1.select("trip_id","trip_start_timestamp","trip_end_timestamp","trip_status","total_fare","total_distance","delay_start_time_mins","customer_id",'driver_id')

In [0]:
df2.show()

+-------+--------------------+--------------------+-----------+----------+--------------+---------------------+-----------+---------+
|trip_id|trip_start_timestamp|  trip_end_timestamp|trip_status|total_fare|total_distance|delay_start_time_mins|customer_id|driver_id|
+-------+--------------------+--------------------+-----------+----------+--------------+---------------------+-----------+---------+
|      1| 2022-12-04 05:17:52| 2022-12-04 07:17:52|  Completed|         0|             0|                    0|          1|       A1|
|      2|2022-12-04 07:17:...|2022-12-04 09:17:...|  Completed|      1949|           222|                    4|          2|       A2|
|      3|2022-12-04 09:17:...|2022-12-04 11:17:...|  Completed|       722|           354|                    0|          3|       A3|
|      4|2022-12-04 11:17:...|2022-12-04 13:17:...|  Completed|       627|           189|                    3|          4|       A4|
|      5|2022-12-04 13:17:...|2022-12-04 15:17:...|  Completed

In [0]:
df2=df2.withColumn("Trip_Date",F.expr("to_date(trip_start_timestamp)"))

In [0]:
df2.show()

+-------+--------------------+--------------------+-----------+----------+--------------+---------------------+-----------+---------+----------+
|trip_id|trip_start_timestamp|  trip_end_timestamp|trip_status|total_fare|total_distance|delay_start_time_mins|customer_id|driver_id| Trip_Date|
+-------+--------------------+--------------------+-----------+----------+--------------+---------------------+-----------+---------+----------+
|      1| 2022-12-04 05:17:52| 2022-12-04 07:17:52|  Completed|         0|             0|                    0|          1|       A1|2022-12-04|
|      2|2022-12-04 07:17:...|2022-12-04 09:17:...|  Completed|      1949|           222|                    4|          2|       A2|2022-12-04|
|      3|2022-12-04 09:17:...|2022-12-04 11:17:...|  Completed|       722|           354|                    0|          3|       A3|2022-12-04|
|      4|2022-12-04 11:17:...|2022-12-04 13:17:...|  Completed|       627|           189|                    3|          4|       

In [0]:
file_path4 = "silver_Zone/Trip_Transactions_Fact"

In [0]:
df2.repartition(1).write.format("delta").save(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path4}")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1162326114157494>:1
----> 1 df2.repartition(1).write.format("delta").save(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path4}")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1397, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1395     self._jwrite.save()
   1396 else:
-> 1397     self._jwrite.save(path)

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321

#Incremental loads

In [0]:
df_upsert=spark.read.load(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path1}")
df_upsert=df_upsert.withColumn("trip_Date",F.expr("to_date(trip_start_timestamp)"))
df_upsert=df_upsert.filter(df_upsert.trip_Date>=F.expr("Current_date-2"))
df_upsert=df_upsert.select("trip_id","trip_start_timestamp","trip_end_timestamp","trip_status","total_fare","total_distance","delay_start_time_mins","customer_id","driver_id","trip_date")

In [0]:
df_upsert.show(10)

+-------+--------------------+------------------+-----------+----------+--------------+---------------------+-----------+---------+---------+
|trip_id|trip_start_timestamp|trip_end_timestamp|trip_status|total_fare|total_distance|delay_start_time_mins|customer_id|driver_id|trip_date|
+-------+--------------------+------------------+-----------+----------+--------------+---------------------+-----------+---------+---------+
+-------+--------------------+------------------+-----------+----------+--------------+---------------------+-----------+---------+---------+



In [0]:
deltaTable = DeltaTable.forPath(spark, f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path4}")

In [0]:
deltaTable.alias("trip_transactions_base").merge(
              source = df_upsert.alias("updates"),
              condition = "trip_transactions_base.trip_id = updates.trip_id"
            ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [0]:
post_merge_count = deltaTable.toDF().count()
print(f"Result: {post_merge_count}")

Result: 2005


In [0]:
spark.read.load(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path4}").select("Trip_Date")

Out[23]: DataFrame[Trip_Date: date]